回归问题：使用movielens数据集中的特征预测该用户给目标电影的评分评分（ratings）：

* 用户观看电影序列 sequence of movie_id（固定长度）。
* 用户对其观看过的电影的评分序列 sequence of ratings。
* 用户画像：`user_id`，`sex`，`occupation` 和 `age_group`。
* 输入序列中每部电影和目标电影的 `类型（genres）` 。
* 预测目标 target_movie_id 的ratings。

修改方式：
* 我们将电影特征（类型）合并到输入序列的每个电影和目标电影的嵌入处理中，而不是将它们视为transformer外部的“other_feature”。
* 我们利用输入序列中的电影评分以及它们在序列中的位置来更新电影，然后再将它们输入到self-attention中。

# 加载数据

In [34]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
! unzip -o ml-1m.zip

--2021-04-14 02:04:32--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  26.3MB/s    in 0.2s    

2021-04-14 02:04:33 (26.3 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [35]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [36]:
users = pd.read_csv('./ml-1m/users.dat', sep='::', names=['user_id', 'sex', 'age_group', 'occupation','zip_code'])
ratings = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['user_id', 'movie_id', 'rating', 'unix_timestamp'])
movies = pd.read_csv('./ml-1m/movies.dat', sep="::", names=['movie_id', 'title', 'genres'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel 

In [37]:
users.head(1)

,user_id,sex,age_group,occupation,zip_code
0,1,F,1,10,48067


In [38]:
ratings.head(1)

,user_id,movie_id,rating,unix_timestamp
0,1,1193,5,978300760


In [39]:
movies.head(1)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy


In [40]:
# 合并为一个大DataFrame
matrix = pd.merge(users, ratings, on='user_id', how='left')
matrix = pd.merge(matrix, movies, on='movie_id', how='left')

# 性别特征做0-1编码
temp = pd.get_dummies(matrix['sex'], prefix = 'sex')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop('sex', axis=1, inplace=True)

In [41]:
# gender_genres = genres
# gender_genres.append('sex_F')
# gender_genres.append('sex_M')
# matrix[genres + str('sex_F') + str('sex+M')]#.head(1)

## 特征工程

### 单标签编码：movie_id、age_group、rating、occupation

In [42]:
# from sklearn.preprocessing import LabelEncoder
# for i in ['user_id', 'age_group', 'occupation']:
#     users[i] = LabelEncoder().fit_transform(users[i])

# for i in ['movie_id', 'user_id', 'rating']:
#     ratings[i] = LabelEncoder().fit_transform(ratings[i])

# movies['movie_id'] = LabelEncoder().fit_transform(movies['movie_id'])

In [43]:
users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
users["age_group"] = users["age_group"].apply(lambda x: f"group_{x}")
users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")

ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

这里不使用LabelEncoder，否则后面再处理 DataFrame中list变为 str 会出错

In [44]:
users.head(1)

,user_id,sex,age_group,occupation,zip_code
0,user_1,F,group_1,occupation_10,48067


In [45]:
movies.head(1)

,movie_id,title,genres
0,movie_1,Toy Story (1995),Animation|Children's|Comedy


In [46]:
ratings.head(1)

,user_id,movie_id,rating,unix_timestamp
0,user_1,movie_1193,5.0,978300760


### 多标签编码：电影类型 genres

In [47]:
genres = ["Action","Adventure", "Animation", "Children's", "Comedy", "Crime",
    "Documentary","Drama","Fantasy", "Film-Noir", "Horror", "Musical","Mystery",
    "Romance","Sci-Fi","Thriller","War","Western"]

for genre in genres: 
    movies[genre] = movies['genres'].apply(lambda x: int(genre in x.split('|')))

In [48]:
movies.head(1)

,movie_id,title,genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,movie_1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


### 特征序列化：电影评分转换为序列

* 我们按照时间（unix_timestamp）对评分进行排序，再将 movie_id 和 ratings 按 user_id分组（groupby)。

* 输出DataFrame是每个user_id已评分电影的序列 和 这些电影的评分。

In [49]:
ratings_group = ratings.sort_values(by=["unix_timestamp"]).groupby("user_id")

ratings_data = pd.DataFrame({"user_id": list(ratings_group.groups.keys()),
                            "movie_ids": list(ratings_group.movie_id.apply(list)),
                            "ratings": list(ratings_group.rating.apply(list)),
                            "timestamps": list(ratings_group.unix_timestamp.apply(list)),}
)

In [50]:
ratings_data.head(2)

,user_id,movie_ids,ratings,timestamps
0,user_1,"[movie_3186, movie_1721, movie_1270, movie_102...","[4.0, 4.0, 5.0, 5.0, 3.0, 5.0, 4.0, 4.0, 5.0, ...","[978300019, 978300055, 978300055, 978300055, 9..."
1,user_10,"[movie_597, movie_858, movie_743, movie_1210, ...","[4.0, 3.0, 3.0, 4.0, 4.0, 5.0, 5.0, 5.0, 3.0, ...","[978224375, 978224375, 978224375, 978224400, 9..."


### 构建数据集：用户看过电影id每4个做一组，用户评分每4个做一组

* 将 `movie_ids` 列表分成一组固定长度的序列，`ratings`也是如此。 设置`sequence_length` 变量就可以更改输入序列的长度；更改`step_size`以控制要为每个用户生成的sequences。

* 业务场景说明序列越靠近现今价值越大，所以我们选择用户点击序列的倒数前4个。阿里巴巴DIEN中是使用最近200个序列固定序列长度，这种方法也让重度电影爱好者和轻度电影爱好者具有相同的权重，预测过程中模型能对 重度使用者 和 轻度使用使用者一视同仁。

In [51]:
# movie_ids[-sequence_length: ] # 最近200个用户点击行为的价值更大
sequence_length = 4
step_size = 2

def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


ratings_data.movie_ids = ratings_data.movie_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

ratings_data.ratings = ratings_data.ratings.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

del ratings_data["timestamps"]

In [52]:
# 每个用户看过电影序列的序列 和 看过电影的评分序列
ratings_data.head(2)

,user_id,movie_ids,ratings
0,user_1,"[[movie_3186, movie_1721, movie_1270, movie_10...","[[4.0, 4.0, 5.0, 5.0], [5.0, 5.0, 3.0, 5.0], [..."
1,user_10,"[[movie_597, movie_858, movie_743, movie_1210]...","[[4.0, 3.0, 3.0, 4.0], [3.0, 4.0, 4.0, 5.0], [..."


让每个序列在DataFrame中独立成行。 

将user feature与ratings数据结合在一起。

In [53]:
ratings_data_movies = ratings_data[["user_id", "movie_ids"]].explode(
    "movie_ids", ignore_index=True
)
ratings_data_rating = ratings_data[["ratings"]].explode("ratings", ignore_index=True)
ratings_data_transformed = pd.concat([ratings_data_movies, ratings_data_rating], axis=1)
ratings_data_transformed = ratings_data_transformed.join(
    users.set_index("user_id"), on="user_id"
)

In [54]:
ratings_data_movies.head(3)

,user_id,movie_ids
0,user_1,"[movie_3186, movie_1721, movie_1270, movie_1022]"
1,user_1,"[movie_1270, movie_1022, movie_2340, movie_1836]"
2,user_1,"[movie_2340, movie_1836, movie_3408, movie_1207]"


In [55]:
# pandas中的 list --> str : [4.0, 4.0, 5.0, 5.0]  --> 4.0, 4.0, 5.0, 5.0

ratings_data_transformed.movie_ids = ratings_data_transformed.movie_ids.apply(
    lambda x: ",".join(x)
)
ratings_data_transformed.ratings = ratings_data_transformed.ratings.apply(
    lambda x: ",".join([str(v) for v in x])
)

del ratings_data_transformed["zip_code"]

ratings_data_transformed.rename(
    columns={"movie_ids": "sequence_movie_ids", "ratings": "sequence_ratings"},
    inplace=True,
)

In [56]:
ratings_data_transformed.head(2)

,user_id,sequence_movie_ids,sequence_ratings,sex,age_group,occupation
0,user_1,"movie_3186,movie_1721,movie_1270,movie_1022","4.0,4.0,5.0,5.0",F,group_1,occupation_10
1,user_1,"movie_1270,movie_1022,movie_2340,movie_1836","5.0,5.0,3.0,5.0",F,group_1,occupation_10


在sequence_length=4且 step_size=2的情况下，我们得到498,623个序列。

85％数据作为训练集， 15％的数据作为测试集

In [57]:
random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.85
train_data = ratings_data_transformed[random_selection]
test_data = ratings_data_transformed[~random_selection]

train_data.to_csv("train_data.csv", index=False, sep="|", header=False)
test_data.to_csv("test_data.csv", index=False, sep="|", header=False)

In [58]:
# matrix = users.merge(ratings, on='user_id', how='left')
# matrix = matrix.merge(movies, on='movie_id', how='left')

In [59]:
# matrix.head(2)

In [60]:
# 特征工程
# matrix['sex']= pd.get_dummies(matrix['sex'], prefix_sex='sex')
# from sklearn.preprocessing import LabelEncoder
# matrix['sex'] = LabelEncoder().fit_transform(matrix['sex'])

## 定义数据

In [62]:
CSV_HEADER = list(ratings_data_transformed.columns)

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "user_id": list(users.user_id.unique()),
    "movie_id": list(movies.movie_id.unique()),
    "sex": list(users.sex.unique()),
    "age_group": list(users.age_group.unique()),
    "occupation": list(users.occupation.unique()),
}

# 用户画像
USER_FEATURES = ["sex", "age_group", "occupation"]

# 电影画像
MOVIE_FEATURES = ["genres"]

## 通过`tf.data.Dataset`创建训练集(training)和测试集(evaluation)

In [63]:
def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    def process(features):
        movie_ids_string = features["sequence_movie_ids"]
        sequence_movie_ids = tf.strings.split(movie_ids_string, ",").to_tensor()

        # 预测目标是用户观看电影序列的最后一部电影：The last movie id in the sequence is the target movie.
        features["target_movie_id"] = sequence_movie_ids[:, -1]
        features["sequence_movie_ids"] = sequence_movie_ids[:, :-1]

        ratings_string = features["sequence_ratings"]
        sequence_ratings = tf.strings.to_number(
            tf.strings.split(ratings_string, ","), tf.dtypes.float32
        ).to_tensor()

        # 用户观看电影评分序列的最后一部电影作为评分预测的目标：The last rating in the sequence is the target for the model to predict.
        target = sequence_ratings[:, -1]
        features["sequence_ratings"] = sequence_ratings[:, :-1]
        return features, target

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        num_epochs=1,
        header=False,
        field_delim="|",
        shuffle=shuffle,
    ).map(process)

    return dataset

In [64]:
# 测试
train_test= get_dataset_from_csv('train_data.csv', shuffle=True, batch_size=265)

In [65]:
# 显示第一条喂给模型的数据
next(iter(train_test))

(OrderedDict([('user_id', <tf.Tensor: shape=(265,), dtype=string, numpy=
               array([b'user_1000', b'user_1040', b'user_1100', b'user_1069',
                      b'user_1110', b'user_1101', b'user_1015', b'user_1015',
                      b'user_1031', b'user_1082', b'user_1010', b'user_1115',
                      b'user_1117', b'user_1020', b'user_1019', b'user_1029',
                      b'user_1029', b'user_1015', b'user_1017', b'user_1082',
                      b'user_1086', b'user_1109', b'user_1101', b'user_1015',
                      b'user_1117', b'user_1068', b'user_1001', b'user_1017',
                      b'user_1112', b'user_1017', b'user_1015', b'user_1100',
                      b'user_1088', b'user_10', b'user_1034', b'user_1067', b'user_1042',
                      b'user_1100', b'user_1112', b'user_1037', b'user_1084',
                      b'user_1001', b'user_1019', b'user_1032', b'user_100',
                      b'user_1050', b'user_1086', b'user_1

## 创建 model_inputs

In [66]:
def create_model_inputs():
    return {"user_id": tf.keras.layers.Input(name="user_id", shape=(1,), dtype=tf.string),
                "sequence_movie_ids": tf.keras.layers.Input(
                    name="sequence_movie_ids", shape=(sequence_length - 1,), dtype=tf.string
                ),
                "target_movie_id": tf.keras.layers.Input(
                    name="target_movie_id", shape=(1,), dtype=tf.string
                ),
                "sequence_ratings": tf.keras.layers.Input(
                    name="sequence_ratings", shape=(sequence_length - 1,), dtype=tf.float32
                ),
                "sex": tf.keras.layers.Input(name="sex", shape=(1,), dtype=tf.string),
                "age_group": tf.keras.layers.Input(name="age_group", shape=(1,), dtype=tf.string),
                "occupation": tf.keras.layers.Input(name="occupation", shape=(1,), dtype=tf.string),
                # 'sex_genres': tf.keras.layers.Input(name='sex_genres', shape=(1, ), dtype=tf.float16)
            }

## Encode input features

encode_input_features可以这么操作：

* 每个用户特征均使用tf.keras.layer.Embedding进行编码，embedding_dim等于特征个数的平方根取整。

* 电影序列 sequence_movie_ids 中每部电影和目标电影都使用tf.keras.layers.Embedding编码。Embedding_dim是电影数量的平方根取整。

* 将每部电影的embedding与其电影画像Embedding融合起来，并使用非线性激活函数激活，输出有同等大小的张量。

* **将位置embedding和序列中的每个电影Embedding相加，再乘以分级序列中的分级，相当于把使用评分作为权重给电影id加权**。

* 目标电影Embedding与序列影片Embedding进行连接，生成`[batch_size, seq_len, embedding_dim]`的张量。

* 该方法返回两个元素的元组：encoded_transformer_features和encoded_other_features。

In [67]:
def encode_input_features(inputs, include_user_id=True, include_user_features=True,include_movie_features=True):
    encoded_transformer_features = []
    encoded_other_features = []

    other_feature_names = []
    if include_user_id:
        other_feature_names.append("user_id")
    if include_user_features:
        other_feature_names.extend(USER_FEATURES)

    ## 用户画像编码： user features
    for feature_name in other_feature_names:
        # 在tf中将字符串输入转为整数输入
        vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
        idx = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=vocabulary, mask_token=None, num_oov_indices=0)(
            inputs[feature_name]
        )
        # 自动计算embedding_dim的维度
        embedding_dims = int(np.sqrt(len(vocabulary)))
        # 上述特征喂给Embedding
        embedding_encoder = tf.keras.layers.Embedding(
            input_dim=len(vocabulary),
            output_dim=embedding_dims,
            trainable=True,
            name=f"{feature_name}_embedding",
        )
        # 将每个特征的Embedding加入到列表中记录下来
        encoded_other_features.append(embedding_encoder(idx))

    ## 所有用户画像的Embedding拼接成一个向量
    if len(encoded_other_features) > 1:
        encoded_other_features = tf.keras.layers.concatenate(encoded_other_features)
    elif len(encoded_other_features) == 1:
        encoded_other_features = encoded_other_features[0]
    else:
        encoded_other_features = None

    ## 创建电影的Embedding
    movie_vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY["movie_id"]
    movie_embedding_dims = int(np.sqrt(len(movie_vocabulary)))
    # 标签编码：在tf中将输入 movieid_823, movieid_821, ... 转为数字
    movie_index_lookup = tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=movie_vocabulary,
        mask_token=None,
        num_oov_indices=0,
        name="movie_index_lookup",
    )
    # 降维：使用tf.keras.layers.Embedding把输入特征降维为固定维度
    movie_embedding_encoder = tf.keras.layers.Embedding(
        input_dim=len(movie_vocabulary),
        output_dim=movie_embedding_dims,
        trainable=True,
        name=f"movie_embedding",
    )
    # 所有电影画像的Embedding拼接为一个向量
    genre_vectors = movies[genres].to_numpy()
    movie_genres_lookup = tf.keras.layers.Embedding(
        input_dim=genre_vectors.shape[0],
        output_dim=genre_vectors.shape[1],
        embeddings_initializer=tf.keras.initializers.Constant(genre_vectors),
        trainable=False,
        name="genres_vector",
    )

    # 构建特征工程 性别和电影类型交叉特征
    # gender_genres = genres
    # gender_genres.append('sex_F')
    # gender_genres.append('sex_M')
    
    # gender_genre_vectors = matrix[gender_genres].to_numpy()
    # gender_genres_lookup = tf.keras.layers.Embedding(input_dim=gender_genre_vectors.shape[0], 
    #                                                  output_dim=gender_genre_vectors.shape[1],
    #                                                  embeddings_initializer=tf.keras.initializers.Constant(gender_genre_vectors), 
    #                                                  trainabel=False,
    #                                                  name='gender_genre_vectors')

    # 为电影画像(genres)创建一个简单的神经网络学习权重，因为genres embedding的权重不更新(trainabel=False)，所以单独设置神经网络学习权重
    movie_embedding_processor = tf.keras.layers.Dense(
        units=movie_embedding_dims,
        activation="relu",
        name="process_movie_embedding_with_genres",
    )

    ## 定义函数对给定的movie_id进行编码
    def encode_movie(movie_id):
        # 标签编码：在tf中将字符串转为整数
        movie_idx = movie_index_lookup(movie_id)
        # 用Embedding向量表示一个movie_id
        movie_embedding = movie_embedding_encoder(movie_idx)
        encoded_movie = movie_embedding
        if include_movie_features:
            movie_genres_vector = movie_genres_lookup(movie_idx)
            # 电影画像 = movie_id embedding + genres embedding
            encoded_movie = movie_embedding_processor(
                tf.keras.layers.concatenate([movie_embedding, movie_genres_vector])
            )
        return encoded_movie

    ## 编码 target_movie_id
    target_movie_id = inputs["target_movie_id"]
    encoded_target_movie = encode_movie(target_movie_id)

    ## 编码用户观看电影序列 sequence movie_ids.
    sequence_movies_ids = inputs["sequence_movie_ids"]
    encoded_sequence_movies = encode_movie(sequence_movies_ids)
    # 创建序列的位置编码 positional embedding
    position_embedding_encoder = tf.keras.layers.Embedding(
        input_dim=sequence_length,
        output_dim=movie_embedding_dims,
        trainable=True,
        name="position_embedding",
    )

    positions = tf.range(start=0, limit=sequence_length - 1, delta=1)
    encodded_positions = position_embedding_encoder(positions)
    # 找到电影对应的评分序列（sequence ratings）后，跟电影embedding融合者融合
    sequence_ratings = tf.expand_dims(inputs["sequence_ratings"], -1)
    # 给 movie encodings 增加 Positional Embedding，
    # 之后再乘以用户对这些电影的打分
    encoded_sequence_movies_with_poistion_and_rating = tf.keras.layers.Multiply()(
        [(encoded_sequence_movies + encodded_positions), sequence_ratings]
    )

    # 输入Transformer的embedding进行拼接
    for encoded_movie in tf.unstack(encoded_sequence_movies_with_poistion_and_rating, axis=1):
        encoded_transformer_features.append(tf.expand_dims(encoded_movie, 1))
    encoded_transformer_features.append(encoded_target_movie)

    encoded_transformer_features = tf.keras.layers.concatenate(encoded_transformer_features, axis=1)

    return encoded_transformer_features, encoded_other_features
    
# 未来可升级的地方
# 修改为类别特征和连续特征 类别特征喂给FM，dense特征喂给DNN 

## Create a BST model


原始例子中other_features没有喂给Transformer，只喂了sequence

【未完成】所有特征都喂给Transformer

【未完成】other feature 需要喂给FM做特征交叉

In [68]:
include_user_id = False
include_user_features = False
include_movie_features = False

hidden_units = [256, 128] # 使用3层全连接层效果下降，应该是网络层数导致模型发生退化问题
dropout_rate =  0.1
num_heads = 3


def create_model():
    inputs = create_model_inputs()
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_movie_features
    )

    # 输出数据进行归一化
    # transform_features = tf.keras.layers.LayerNormalization()(transformer_features)

    # 搭建 Multi-head Attention 层
    attention_output = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block.
    attention_output = tf.keras.layers.Dropout(dropout_rate)(attention_output)
    # 残差连接
    x1 = tf.keras.layers.Add()([transformer_features, attention_output])
    # 可将layerNormalization 向前调整到输入层
    x1 = tf.keras.layers.LayerNormalization()(x1)
    x2 = tf.keras.layers.LeakyReLU()(x1)
    x2 = tf.keras.layers.Dense(units=x2.shape[-1])(x2)
    x2 = tf.keras.layers.Dropout(dropout_rate)(x2)
    # Add & Norm 残差连接 & 层归一化
    transformer_features = tf.keras.layers.Add()([x1, x2])
    transformer_features = tf.keras.layers.LayerNormalization()(transformer_features)
    features = tf.keras.layers.Flatten()(transformer_features) #(None, 248)

    # 包含其他特征
    # if other_features is not None:
    #     features = tf.keras.layers.concatenate(
    #         [features, tf.keras.layers.Reshape([other_features.shape[-1]])(other_features)]
    #     )

        # concated_embeds_value = tf.keras.layers.Reshape([other_features.shape[-1]])(other_features)
        # square_of_sum = tf.square(tf.reduce_sum(concated_embeds_value, axis=1), keepdims=True)
        # sum_of_square = tf.reduce_sum(tf.pow(concated_embeds_value, 2), axis=1, keepdims=True)  # concated_embeds_value * concated_embeds_value
        # other_features = 0.5 * (square_of_sum - sum_of_square)
        # features = tf.concat([features, other_features], axis=1)

    # Fully-connected layers.
    features_ori = features
    for num_units in hidden_units:
        features = tf.keras.layers.Dense(num_units)(features)
        features = tf.keras.layers.BatchNormalization()(features)
        features = tf.keras.layers.LeakyReLU()(features)
        features = tf.keras.layers.Dropout(dropout_rate)(features)
    
    # outputs = dnn_n_resnet(features)
    # 残差连接
    # features_ori = tf.keras.layers.Dense(units=hidden_units[-1], activation=None, trainable=False)(features_ori)
    # features = tf.concat([features, features_ori], axis=-1)
    # features = features + features_ori
    outputs = tf.keras.layers.Dense(units=1)(features)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = create_model()

In [105]:
include_user_id = False
include_user_features = False
include_movie_features = False

hidden_units = [256, 128, 64] # 使用3层全连接层效果下降，应该是网络层数导致模型发生退化问题
dropout_rate =  0.3
num_heads = 3


def create_model():
    inputs = create_model_inputs()
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_movie_features
    )

    # transformer_features = tf.keras.layers.concatenate([transform_features, other_features])
    # 输出数据进行归一化
    transform_features = tf.keras.layers.LayerNormalization()(transformer_features)

    # 搭建 Multi-head Attention 层
    attention_output = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block.
    attention_output = tf.keras.layers.Dropout(dropout_rate)(attention_output)
    # 残差连接
    x1 = tf.keras.layers.Add()([transformer_features, attention_output])
    # 可将layerNormalization 向前调整到输入层
    x1 = tf.keras.layers.LayerNormalization()(x1)
    x2 = tf.keras.layers.LeakyReLU()(x1)
    x2 = tf.keras.layers.Dense(units=x2.shape[-1])(x2)
    x2 = tf.keras.layers.Dropout(dropout_rate)(x2)
    # Add & Norm 残差连接 & 层归一化
    transformer_features = tf.keras.layers.Add()([x1, x2])
    transformer_features = tf.keras.layers.LayerNormalization()(transformer_features)
    features = tf.keras.layers.Flatten()(transformer_features) #(None, 248)

    # 包含其他特征
    # if other_features is not None:
    #     features = tf.keras.layers.concatenate(
    #         [features, tf.keras.layers.Reshape([other_features.shape[-1]])(other_features)]
    #     )

        # concated_embeds_value = tf.keras.layers.Reshape([other_features.shape[-1]])(other_features)
        # square_of_sum = tf.square(tf.reduce_sum(concated_embeds_value, axis=1), keepdims=True)
        # sum_of_square = tf.reduce_sum(tf.pow(concated_embeds_value, 2), axis=1, keepdims=True)  # concated_embeds_value * concated_embeds_value
        # other_features = 0.5 * (square_of_sum - sum_of_square)
        # features = tf.concat([features, other_features], axis=1)

    # Fully-connected layers.
    features_ori = features
    for num_units in hidden_units:
        features = tf.keras.layers.Dense(num_units)(features)
        features = tf.keras.layers.BatchNormalization()(features)
        features = tf.keras.layers.LeakyReLU()(features)
        features = tf.keras.layers.Dropout(dropout_rate)(features)
    
    # outputs = dnn_n_resnet(features)
    # 残差连接
    # features_ori = tf.keras.layers.Dense(units=hidden_units[-1], activation=None, trainable=False)(features_ori)
    # features = tf.concat([features, features_ori], axis=-1)
    # features = features + features_ori
    outputs = tf.keras.layers.Dense(units=1)(features)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = create_model()

## Run training and evaluation experiment

In [106]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence_movie_ids (InputLayer) [(None, 3)]          0                                            
__________________________________________________________________________________________________
movie_index_lookup (StringLooku multiple             0           target_movie_id[0][0]            
                                                                 sequence_movie_ids[0][0]         
__________________________________________________________________________________________________
movie_embedding (Embedding)     multiple             240746      movie_index_lookup[0][0]         
                                                                 movie_index_lookup[1][0]         
____________________________________________________________________________________________

In [107]:
%%time
# Compile the model.
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()],
)

# Read the training data.
train_dataset = get_dataset_from_csv("train_data.csv", shuffle=True, batch_size=512)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1', patience=3, restore_best_weights=True)

# Fit the model with the training data.
model.fit(train_dataset, epochs=5,
        #   callbacks=[early_stopping],  # checkpoint

          )

# Read the test data.
test_dataset = get_dataset_from_csv("test_data.csv", batch_size=265) # 512

# Evaluate the model on the test data.
_, rmse = model.evaluate(test_dataset, verbose=0)
print(f"Test MAE: {round(rmse, 3)}")
# ===========================Test==============================
# print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])

Epoch 1/5
829/829 [==============================] - 14s 15ms/step - loss: 4.0266 - mean_absolute_error: 1.5569
Epoch 2/5
829/829 [==============================] - 12s 15ms/step - loss: 1.3136 - mean_absolute_error: 0.9152
Epoch 3/5
829/829 [==============================] - 12s 15ms/step - loss: 1.1108 - mean_absolute_error: 0.8433
Epoch 4/5
829/829 [==============================] - 12s 14ms/step - loss: 1.0304 - mean_absolute_error: 0.8106
Epoch 5/5
829/829 [==============================] - 12s 15ms/step - loss: 0.9824 - mean_absolute_error: 0.7920
Test MAE: 0.757
CPU times: user 1min 25s, sys: 13.7 s, total: 1min 39s
Wall time: 1min 4s


In [81]:
ratings

,user_id,movie_id,rating,unix_timestamp
0,user_1,movie_1193,5.0,978300760
1,user_1,movie_661,3.0,978302109
2,user_1,movie_914,3.0,978301968
3,user_1,movie_3408,4.0,978300275
4,user_1,movie_2355,5.0,978824291
...,...,...,...,...
1000204,user_6040,movie_1091,1.0,956716541
1000205,user_6040,movie_1094,5.0,956704887
1000206,user_6040,movie_562,5.0,956704746
1000207,user_6040,movie_1096,4.0,956715648
